In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [64]:
import os
import time
import torch
import torchvision
import torchvision.transforms as transforms
from torchvision.models import resnet50
from torch.utils.data import DataLoader, WeightedRandomSampler
from torchvision.datasets import ImageFolder
from sklearn.model_selection import train_test_split
from torch.optim import Adam
from torch.cuda.amp import GradScaler, autocast


In [71]:
# Data loading and preprocessing
def load_data(data_dir, batch_size, num_workers):
    transform_train = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(10),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    transform_test = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])

    full_dataset = ImageFolder(data_dir)
    train_idx, test_idx = train_test_split(list(range(len(full_dataset))), test_size=0.2, random_state=42)

    train_data = torch.utils.data.Subset(full_dataset, train_idx)
    test_data = torch.utils.data.Subset(full_dataset, test_idx)

    # Apply transforms
    train_data.dataset.transform = transform_train
    test_data.dataset.transform = transform_test

    # Calculate class weights
    class_counts = [0] * 50
    for idx in train_idx:
        class_counts[full_dataset.targets[idx]] += 1
    class_weights = 1. / torch.tensor([x + 1e-6 for x in class_counts], dtype=torch.float)
    sample_weights = [class_weights[target] for target in full_dataset.targets if target in train_idx]

    sampler = WeightedRandomSampler(sample_weights, num_samples=len(sample_weights), replacement=True)

    train_loader = DataLoader(train_data, batch_size=batch_size, sampler=sampler, num_workers=num_workers, pin_memory=True)
    test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False, num_workers=num_workers, pin_memory=True)

    return train_loader, test_loader



In [72]:
# Model definition
def create_model(num_classes):
    model = resnet50(pretrained=True)
    num_ftrs = model.fc.in_features
    model.fc = torch.nn.Sequential(
        torch.nn.BatchNorm1d(num_ftrs),
        torch.nn.Dropout(0.5),
        torch.nn.Linear(num_ftrs, num_classes)
    )
    return model


In [73]:
# Training loop
def train_model(model, train_loader, device, epochs=10, lr=0.001):
    model = model.to(device)
    criterion = torch.nn.CrossEntropyLoss()
    optimizer = Adam(model.parameters(), lr=lr)
    scaler = GradScaler()
    start_time = time.time()

    for epoch in range(epochs):
        model.train()
        for i, (inputs, labels) in enumerate(train_loader):
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()

            with autocast():
                outputs = model(inputs)
                loss = criterion(outputs, labels)

            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()

            if (i + 1) % 20 == 0:
                print(f"Epoch [{epoch+1}/{epochs}], Step [{i+1}/{len(train_loader)}], Loss: {loss.item():.4f}")

    training_time = time.time() - start_time
    print(f"Training completed in {training_time:.2f} seconds.")

In [74]:
# Evaluation function
def evaluate_model(model, test_loader, device):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    accuracy = 100 * correct / total
    print('Test Accuracy: {:.2f}%'.format(accuracy))


In [75]:

# Main
if __name__ == "__main__":
    data_dir = '/content/drive/MyDrive/JPEGImages'
    batch_size = 64
    num_workers = 8
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    train_loader, test_loader = load_data(data_dir, batch_size, num_workers)
    model = create_model(50)
    train_model(model, train_loader, device)
    evaluate_model(model, test_loader, device)


Epoch [1/10], Step [20/115], Loss: 2.8278
Epoch [1/10], Step [40/115], Loss: 2.1973
Epoch [1/10], Step [60/115], Loss: 1.7469
Epoch [1/10], Step [80/115], Loss: 1.4204
Epoch [1/10], Step [100/115], Loss: 1.6971
Epoch [2/10], Step [20/115], Loss: 1.3126
Epoch [2/10], Step [40/115], Loss: 1.1029
Epoch [2/10], Step [60/115], Loss: 1.0474
Epoch [2/10], Step [80/115], Loss: 0.8008
Epoch [2/10], Step [100/115], Loss: 1.3350
Epoch [3/10], Step [20/115], Loss: 0.8101
Epoch [3/10], Step [40/115], Loss: 0.9621
Epoch [3/10], Step [60/115], Loss: 0.6383
Epoch [3/10], Step [80/115], Loss: 0.6198
Epoch [3/10], Step [100/115], Loss: 0.4094
Epoch [4/10], Step [20/115], Loss: 0.7007
Epoch [4/10], Step [40/115], Loss: 0.5796
Epoch [4/10], Step [60/115], Loss: 0.4829
Epoch [4/10], Step [80/115], Loss: 0.6087
Epoch [4/10], Step [100/115], Loss: 0.7161
Epoch [5/10], Step [20/115], Loss: 0.7859
Epoch [5/10], Step [40/115], Loss: 0.5852
Epoch [5/10], Step [60/115], Loss: 0.6573
Epoch [5/10], Step [80/115], L